In [100]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(100_000_000_000)

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils
import utils

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt

import rt_analysis as rt
import importlib

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

In [2]:
analysis_date = dt.date(2022, 2, 8)

In [3]:
samtrans_itp_id = 290

In [58]:
importlib.reload(utils)
importlib.reload(rt)

<module 'rt_analysis' from '/home/jovyan/data-analyses/rt_delay/rt_analysis.py'>

In [5]:
pbar = tqdm()

0it [00:00, ?it/s]

In [6]:
sam = rt.OperatorDayAnalysis(samtrans_itp_id, analysis_date, pbar)

found parquet
found parquet
found parquet
found parquet
found_parquet
vehicle positions gdf must not be empty
vehicle positions gdf must not be empty
vehicle positions gdf must not be empty
vehicle positions gdf must not be empty
vehicle positions gdf must not be empty
vehicle positions gdf must not be empty
vehicle positions gdf must not be empty
vehicle positions gdf must not be empty
vehicle positions gdf must not be empty
vehicle positions gdf must not be empty
vehicle positions gdf must not be empty
vehicle positions gdf must not be empty
vehicle positions gdf must not be empty
vehicle positions gdf must not be empty
vehicle positions gdf must not be empty
vehicle positions gdf must not be empty
vehicle positions gdf must not be empty
vehicle positions gdf must not be empty
vehicle positions gdf must not be empty
vehicle positions gdf must not be empty
vehicle positions gdf must not be empty
vehicle positions gdf must not be empty
vehicle positions gdf must not be empty
vehicle po

/opt/conda/lib/python3.9/site-packages/siuba/sql/utils.py:52: SAWarning: Dialect bigquery:bigquery will not make use of SQL compilation caching as it does not set the 'supports_statement_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Dialect maintainers should seek to set this attribute to True after appropriate development and testing for SQLAlchemy 1.4 caching support.   Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/14/cprf)


In [32]:
(sam.stop_delay_view
 >> group_by(_.stop_id, _.stop_name)
 >> summarize(mean_delay = _.delay.mean(), max_delay = _.delay.max())
 >> arrange(-_.mean_delay)) >> head(10)

,stop_id,stop_name,mean_delay,max_delay
1298,343118,Alameda de las Pulgas & Alma St,0 days 00:21:15.454282,0 days 00:28:24.245235
1574,346126,Terminal Ave & Almanor Ave,0 days 00:20:07.792832,0 days 00:20:07.792832
765,334620,South SF BART-Bay 5 Westside Busway,0 days 00:19:56.999178800,0 days 00:31:08.739457
1625,346596,Hamilton Ave & Henderson Ave,0 days 00:19:25.675944,0 days 00:19:25.675944
1626,346597,Hamilton Ave & Hollyburne Ave,0 days 00:18:59.449394,0 days 00:18:59.449394
1623,346594,Kavanaugh Dr & Kirkwood Ct,0 days 00:17:28.461803,0 days 00:17:28.461803
1811,363044,Notre Dame Ave & Illinois St,0 days 00:16:31.244881,0 days 00:16:31.244881
1622,346593,Kavanaugh Dr & Farrington Way,0 days 00:16:19.299827,0 days 00:16:19.299827
291,331100,SF Transit Center - BART shuttle connection,0 days 00:16:16.500606,0 days 00:16:16.500606
356,332046,Pope St & Bellevue Ave,0 days 00:16:03.956056,0 days 00:16:03.956056


In [8]:
sam.set_filter(start_time='15:00', end_time='19:00')

{'start_time': datetime.time(15, 0), 'end_time': datetime.time(19, 0), 'route_ids': None, 'direction_id': None, 'direction': None}


In [9]:
m = sam.segment_speed_map()

view filter: {'start_time': datetime.time(15, 0), 'end_time': datetime.time(19, 0), 'route_ids': None, 'direction_id': None, 'direction': None}
2960292_1_ empty!
SamTrans80:2960292_0_ empty!
SamTrans80:2810273_0_ empty!
2810273_1_ empty!
ECR1063_0_ empty!
SamTrans80:ECR1063_0_ empty!
2860095_1_ empty!
SamTrans80:2860095_0_ empty!
540084_0_ empty!
SamTrans80:170516_1_ empty!
SamTrans80:180039_0_ empty!
170515_1_ empty!
180040_1_ empty!
SamTrans80:1100237_0_ empty!
speed above 80 for shape SamTrans80:1100237, dropping
SamTrans80:1100256_0_ empty!
negative speed for shape 2960306, dropping
negative speed for shape 2960306, dropping
SamTrans80:2810269_1_ empty!
2810269_0_ empty!
SamTrans80:840049_1_ empty!
SamTrans80:830110_0_ empty!
SamTrans80:820056_1_ empty!
350056_0_ empty!
1220565_1_ empty!
SamTrans80:1220565_1_ empty!
280062_1_ empty!
350093_0_ empty!
2600356_1_ empty!
SamTrans80:2600358_1_ empty!
2600357_1_ empty!
SamTrans80:2600357_1_ empty!
ECR1072_1_ empty!
SamTrans80:ECR1072_1_ 

/opt/conda/lib/python3.9/site-packages/numpy/lib/function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
/opt/conda/lib/python3.9/site-packages/numpy/lib/function_base.py:4009: RuntimeWarning: invalid value encountered in subtract


speed above 80 for shape 3980272, dropping
negative speed for shape 3980272, dropping
3980272_1_ empty!
SamTrans80:3980272_0_ empty!
speed above 80 for shape SamTrans80:3980272, dropping
negative speed for shape SamTrans80:3980272, dropping
SamTrans80:140106_1_ empty!
speed above 80 for shape 1300317, dropping
1300317_1_ empty!
SamTrans80:1300317_1_ empty!
SamTrans80:1210346_1_ empty!
1210331_0_ empty!
2950425_1_ empty!
SamTrans80:2950427_0_ empty!
negative speed for shape SamTrans80:2950427, dropping
negative speed for shape SamTrans80:610093, dropping
SamTrans80:610093_1_ empty!
610093_0_ empty!
negative speed for shape 610093, dropping
negative speed for shape SamTrans80:850086, dropping
SamTrans80:850086_1_ empty!
720063_1_ empty!
2500415_1_ empty!
SamTrans80:1120161_1_ empty!
1120161_1_ empty!
1120180_1_ empty!
590072_1_ empty!
SamTrans80:1210348_0_ empty!
1210348_1_ empty!
negative speed for shape SamTrans80:810070, dropping
SamTrans80:810070_1_ empty!
SamTrans80:810066_1_ empty!

TopologyException: assigned depths do not match at -217628.67881293956 -35629.731197568181
TopologyException: assigned depths do not match at -206250.92608763179 -51291.043813443037
TopologyException: assigned depths do not match at -217628.67881293956 -35629.731197568181
TopologyException: assigned depths do not match at -217628.67881293956 -35629.731197568181
TopologyException: assigned depths do not match at -206250.92608763179 -51291.043813443037
/opt/conda/lib/python3.9/site-packages/shapely/ops.py:634: ShapelyDeprecationWarning: GeometryTypeError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
/home/jovyan/data-analyses/rt_delay/rt_analysis.py:580: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



In [37]:
# m

In [19]:
sam.vehicle_positions >> count(_.trip_id) >> arrange(_.n)

,trip_id,n
1402,3971004,1
1403,3971005,1
1404,3971006,1
1393,11463589-137-Blocks-Weekday-14,6
1398,11463618-137-Blocks-Weekday-14,11
...,...,...
860,11403512-137-Blocks-Weekday-14,519
1046,11426576-137-Blocks-Weekday-14,526
857,11403509-137-Blocks-Weekday-14,531
1073,11426605-137-Blocks-Weekday-51,576


In [11]:
sam.pct_trips_valid_rt

0.5894955079474775

In [34]:
sam.position_interpolators['11401903-137-Blocks-Weekday-14']['rt'].detailed_speed_map()

speed above 80 for trip -8642152625521676425, dropping


/home/jovyan/data-analyses/rt_delay/rt_analysis.py:109: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



In [35]:
sam.position_interpolators['11401833-137-Blocks-Weekday-14']['rt'].detailed_speed_map()

speed above 80 for trip 5247144641311058469, dropping


/home/jovyan/data-analyses/rt_delay/rt_analysis.py:109: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



### info sandbox


In [36]:
ac = rt.OperatorDayAnalysis(4, analysis_date, pbar)

found parquet
found parquet
found parquet
found parquet
found_parquet
vehicle positions gdf must not be empty
vehicle positions gdf must not be empty
vehicle positions gdf must not be empty
less than 1km of data
vehicle positions gdf must not be empty
vehicle positions gdf must not be empty
vehicle positions gdf must not be empty
vehicle positions gdf must not be empty
vehicle positions gdf must not be empty
less than 1km of data
vehicle positions gdf must not be empty
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
vehicle positions gdf must not be empty
vehicle positions gdf must not be empty
less than 1km of data
vehicle positions gdf must not be empty
vehicle positions gdf must not be empty
less than 1km of data
less than 1km 

In [38]:
ac.pct_trips_valid_rt

0.9539436882341973

In [39]:
sam.stop_segment_speed_view

,stop_id,stop_name,geometry,shape_id,shape_meters,trip_key,trip_id,stop_sequence,route_id,direction_id,...,last_loc,meters_from_last,speed_from_last,delay_chg_sec,delay_sec,speed_mph,n_trips,avg_mph,_20p_mph,trips_per_hour
0,363604,Pulgas Ave & Weeks St,"LINESTRING (-188537.213 -58829.505, -188537.21...",2960292,1704.354905,-6559798944168529770,11402652-137-Blocks-Weekday-14,5,296-196,0,...,1044.135052,660.219854,12.226294,-18.0,256,27.350219,2,48.839676,35.946002,0.50
1,363604,Pulgas Ave & Weeks St,"LINESTRING (-188537.213 -58829.505, -188537.21...",2960292,1704.354905,6404310003860338346,11402645-137-Blocks-Weekday-14,5,296-196,0,...,1044.135052,660.219854,31.439041,-51.0,38,70.329134,2,48.839676,35.946002,0.50
2,363605,Bay Rd & Pulgas Ave,"LINESTRING (-188143.923 -58576.715, -188175.25...",2960292,2075.618901,-6559798944168529770,11402652-137-Blocks-Weekday-14,6,296-196,0,...,1704.354905,371.263996,13.259428,-12.0,245,29.661341,3,50.534137,29.661341,0.75
3,363605,Bay Rd & Pulgas Ave,"LINESTRING (-188143.923 -58576.715, -188175.25...",2960292,2075.618901,-751040685652728346,11402654-137-Blocks-Weekday-14,6,296-196,0,...,1704.354905,371.263996,13.259428,-12.0,23,29.661341,3,50.534137,29.661341,0.75
4,363605,Bay Rd & Pulgas Ave,"LINESTRING (-188143.923 -58576.715, -188175.25...",2960292,2075.618901,6404310003860338346,11402645-137-Blocks-Weekday-14,6,296-196,0,...,1704.354905,371.263996,41.251555,-31.0,7,92.279729,3,50.534137,29.661341,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,341111,N Delaware St & Tilton Ave,"LINESTRING (-204710.549 -47326.946, -204762.98...",530070,6049.927298,-5712585560615953286,11428028-137-Blocks-Weekday-14-SDon-1101100,8,53-196,0,...,5642.049015,407.878283,4.582902,10.0,452,10.251952,2,13.574344,11.580909,0.50
9,341111,N Delaware St & Tilton Ave,"LINESTRING (-204710.549 -47326.946, -204762.98...",530070,6049.927298,-1091077376320784811,11427911-137-Blocks-Weekday-14-SDon-1101100,8,53-196,0,...,5642.049015,407.878283,7.553302,-25.0,703,16.896736,2,13.574344,11.580909,0.50
10,341109,N Delaware St & E Bellevue Ave,"LINESTRING (-204969.929 -47012.197, -204970.01...",530070,6826.523056,-5712585560615953286,11428028-137-Blocks-Weekday-14-SDon-1101100,9,53-196,0,...,6049.927298,776.595758,6.471631,-30.0,423,14.477039,2,13.230368,12.482365,0.50
11,341109,N Delaware St & E Bellevue Ave,"LINESTRING (-204151.858 -48004.087, -204183.86...",530070,6826.523056,2131095435240914850,11427911-137-Blocks-Weekday-14-SDon-1101100,9,53-196,0,...,4764.063049,2062.460007,5.357039,-14.0,716,11.983696,2,13.230368,12.482365,0.50


In [40]:
sam.stop_delay_view

,stop_id,stop_name,geometry,shape_id,shape_meters,trip_key,trip_id,stop_sequence,arrival_time,route_id,direction_id,actual_time,delay
0,344207,Middlefield Rd & Flynn Ave,POINT (-195546.323 -57465.009),3970501,14395.523676,8558647170717006042,11403511-137-Blocks-Weekday-14,35,2022-02-08 02:12:58,397-196,0,2022-02-08 02:13:57.352073,0 days 00:00:59.352073
2,334012,S Airport Blvd & Utah Ave,POINT (-211880.540 -38535.159),3970501,45801.636621,8558647170717006042,11403511-137-Blocks-Weekday-14,97,2022-02-08 03:09:58,397-196,0,2022-02-08 03:15:59.333791,0 days 00:06:01.333791
3,341136,El Camino Real & 37th Ave,POINT (-202445.520 -51124.983),3970501,24661.415240,-5293927597646027685,11403511-137-Blocks-Weekday-14,58,2022-02-08 02:30:17,397-196,0,2022-02-08 02:42:17.697381,0 days 00:12:00.697381
4,335508,McDonnell Rd & San Bruno Ave,POINT (-211327.917 -40034.258),3970501,44149.304366,-5293927597646027685,11403511-137-Blocks-Weekday-14,94,2022-02-08 03:06:44,397-196,0,2022-02-08 03:12:46.763966,0 days 00:06:02.763966
5,363005,Bay Rd & University Ave,POINT (-189005.220 -58416.632),3970501,5270.793849,-5293927597646027685,11403511-137-Blocks-Weekday-14,14,2022-02-08 01:55:00,397-196,0,2022-02-08 01:58:01.857318,0 days 00:03:01.857318
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35089,341099,S Delaware St & 16th Ave,POINT (-203718.743 -48517.164),530070,4092.558359,-1091077376320784811,11427911-137-Blocks-Weekday-14-SDon-1101100,5,2022-02-08 15:07:53,53-196,0,2022-02-08 15:20:44.792842,0 days 00:12:51.792842
35091,341608,S Delaware St & Charles Ln,POINT (-203527.337 -48748.892),530070,3791.905092,-1091077376320784811,11427911-137-Blocks-Weekday-14-SDon-1101100,4,2022-02-08 15:06:56,53-196,0,2022-02-08 15:19:20.891670,0 days 00:12:24.891670
35092,341104,S Delaware St & 10th Ave,POINT (-204146.982 -48000.211),530070,4764.063049,2131095435240914850,11427911-137-Blocks-Weekday-14-SDon-1101100,6,2022-02-08 15:10:00,53-196,0,2022-02-08 15:22:09.403669,0 days 00:12:09.403669
35093,341171,Ginniver St & 19th Ave,POINT (-203076.523 -49186.186),530070,2881.682176,-1091077376320784811,11427911-137-Blocks-Weekday-14-SDon-1101100,2,2022-02-08 15:04:04,53-196,0,2022-02-08 15:17:15.399469,0 days 00:13:11.399469


In [41]:
sam.endpoint_delay_view

,stop_id,stop_name,geometry,shape_id,shape_meters,trip_key,trip_id,stop_sequence,arrival_time,route_id,direction_id,actual_time,delay,arrival_hour,mean_speed_mph
0,331072,Main St & Market St,POINT (-210656.317 -22223.451),3970501,66299.896192,8558647170717006042,11403511-137-Blocks-Weekday-14,130,2022-02-08 03:53:01,397-196,0,2022-02-08 03:47:24.485250,0 days 00:00:00,3,20.737849
1,344085,El Camino Real & Jefferson Ave,POINT (-197021.030 -56916.261),2960292,15450.602754,535481820082395133,11402638-137-Blocks-Weekday-14,38,2022-02-08 14:04:01,296-196,0,2022-02-08 14:05:17.640141,0 days 00:01:16.640141,14,12.656018
2,344085,El Camino Real & Jefferson Ave,POINT (-197021.030 -56916.261),2960292,15450.602754,6792351121202089070,11402621-137-Blocks-Weekday-14,38,2022-02-08 08:30:52,296-196,0,2022-02-08 08:40:05.430444,0 days 00:09:13.430444,8,9.910339
3,344085,El Camino Real & Jefferson Ave,POINT (-197021.030 -56916.261),2960292,15450.602754,-6293308567175608532,11402624-137-Blocks-Weekday-14,38,2022-02-08 09:27:01,296-196,0,2022-02-08 09:22:59.284380,0 days 00:00:00,9,13.538748
4,344085,El Camino Real & Jefferson Ave,POINT (-197021.030 -56916.261),2960292,15450.602754,-5087752167477984255,11402634-137-Blocks-Weekday-14,38,2022-02-08 12:40:01,296-196,0,2022-02-08 12:50:32.055050,0 days 00:10:31.055050,12,6.988001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
841,331100,SF Transit Center - BART shuttle connection,POINT (-210775.080 -22484.554),7130029,27841.242531,-1750500544361370066,11403245-137-Blocks-Weekday-14,6,2022-02-08 04:40:00,713-196,0,2022-02-08 04:56:16.500606,0 days 00:16:16.500606,4,20.435052
842,335620,SFO Airport Terminal G-Lower Level,POINT (-210687.660 -41870.684),SamTrans80:7130028,24557.770008,362732395418560750,11403246-137-Blocks-Weekday-14,5,2022-02-08 05:25:00,713,0,2022-02-08 05:21:51.022181,0 days 00:00:00,5,33.541988
843,345901,Marlborough Ave & Dumbarton Ave,POINT (-195407.510 -58250.282),SamTrans80:720009,394.070560,2308550794331696662,11428038-137-Blocks-Weekday-14-SDon,2,2022-02-08 07:53:00,72,0,2022-02-08 07:52:00.924368,0 days 00:00:00,7,16.942739
844,341109,N Delaware St & E Bellevue Ave,POINT (-205461.583 -46411.112),530070,6826.523056,-5712585560615953286,11428028-137-Blocks-Weekday-14-SDon-1101100,9,2022-02-08 15:17:39,53-196,0,2022-02-08 15:24:42.693936,0 days 00:07:03.693936,15,9.982437


In [42]:
sam.endpoint_delay_summary

,direction_id,route_id,arrival_hour,n_trips,mean_end_delay
0,0,110,7,1,0 days 00:01:10.995985
1,0,110,9,1,0 days 01:28:51.788956
2,0,110,12,1,0 days 00:03:01.995940
3,0,110,14,1,0 days 00:01:42.980354
4,0,110,17,1,0 days 00:03:27.460325
...,...,...,...,...,...
692,1,SFO-196,13,1,0 days 00:03:24.326552
693,1,SFO-196,15,1,0 days 00:05:58.287237
694,1,SFO-196,18,1,0 days 00:02:00.662834
695,1,SFO-196,20,1,0 days 00:02:23.917012


In [44]:
sam.route_coverage_summary()

,trip_key,trip_id,shape_id,direction_id,min_meters,min_stop,max_meters,max_stop
0,-9222400322002530974,11402033-137-Blocks-Weekday-14,SamTrans80:1300320,1,3090.616969,5,19129.727540,37
1,-9219796520914221443,11403736-137-Blocks-Weekday-14,SamTrans80:1100236,0,3511.340133,6,16914.280872,34
2,-9216288312626665887,11403742-137-Blocks-Weekday-14,1100236,1,1661.100687,4,17461.833564,35
3,-9190047826441265210,11441233-137-Blocks-Weekday-14,1220566,1,2358.254212,4,23195.083541,52
4,-9178327147980555411,11403087-137-Blocks-Weekday-14,SamTrans80:1200295,0,2415.959982,4,13222.816033,29
...,...,...,...,...,...,...,...,...
1680,9209262428430303052,11403653-137-Blocks-Weekday-14,170521,0,3662.502879,6,20903.143492,40
1681,9214530759959633404,11402662-137-Blocks-Weekday-14,2960292,0,821.907439,3,14641.831012,37
1682,9220612263421195194,11401882-137-Blocks-Weekday-14,SamTrans80:2810273,1,802.726727,2,9775.206954,27
1683,9220624463015121334,11427976-137-Blocks-Weekday-14-SDon,350056,1,1704.861656,4,6853.052508,14


In [80]:
def categorize_cleaning(rt_operator_day, interpolator_key):
    rt_interpolator = rt_operator_day.position_interpolators[interpolator_key]['rt']
    raw = rt_interpolator.position_gdf.shape[0]
    same_loc_dropped = (rt_interpolator.position_gdf >> distinct(_.shape_meters)).shape[0]
    cleaned = rt_interpolator.cleaned_positions.shape[0]
    return (interpolator_key, cleaned / raw, cleaned / same_loc_dropped)

In [109]:
ex = sam.position_interpolators['11454049-137-Blocks-Weekday-14']['rt']

In [110]:
distinct_positions = ex.position_gdf >> distinct(_.shape_meters, _keep_all=True)

In [128]:
import folium

In [129]:
marker = folium.CircleMarker(radius = 3, # Radius in metres
                       weight = 0, #outline weight
                       fill_color = '#000000', 
                       fill_opacity = 1)

In [132]:
def layer_points(rt_interpolator):
    initial_bk_noise = (ex.position_gdf
                        >> filter(_.meters_from_last < 0)
                        >> select(_.geometry, _.shape_meters, _.progressed,
                                  _.secs_from_last, _.meters_from_last)
                       )
    initial_deduped = (ex.position_gdf
                       >> distinct(_.shape_meters, _keep_all=True)
                       >> select(_.geometry, _.shape_meters, _.progressed,
                                 _.secs_from_last, _.meters_from_last)
                      )
    cleaned = ex.cleaned_positions >> select(_.geometry, _.shape_meters, _.progressed,
                                             _.secs_from_last, _.meters_from_last)
    popup_dict = {'shape_meters': 'shape_meters', 'progressed': 'progressed', 'secs_from_last': 'secs_from_last',
                 'meters_from_last': 'meters_from_last'}
    layers_dict = {'initial backwards noise': {'df': initial_bk_noise,
                                              'plot_col': 'shape_meters',
                                              'popup_dict': popup_dict,
                                              'tooltip_dict': popup_dict,
                                              'colorscale': branca.colormap.step.Blues_03},
                   'initial position deduplicated': {'df': initial_deduped,
                                              'plot_col': 'shape_meters',
                                              'popup_dict': popup_dict,
                                              'tooltip_dict': popup_dict,
                                              'colorscale': branca.colormap.step.Greens_03},
                   'cleaned': {'df': cleaned,
                                              'plot_col': 'shape_meters',
                                              'popup_dict': popup_dict,
                                              'tooltip_dict': popup_dict,
                                              'colorscale': branca.colormap.step.Reds_03,
                                              'marker':  marker},
                  }
    return shared_utils.map_utils.make_folium_multiple_layers_map(layers_dict, 1200, 700, marker=marker)

In [133]:
layer_points(ex)

In [120]:
ex.position_gdf >> filter(_.meters_from_last < 0)

,header_timestamp,vehicle_timestamp,vehicle_longitude,vehicle_latitude,geometry,shape_meters,secs_from_last,meters_from_last,progressed,speed_from_last
57,2022-02-08 12:23:36,2022-02-08 12:23:11,-122.410614,37.630960,POINT (-212426.232 -40143.509),21592.338026,31.0,-3850.815701,False,-124.219861
84,2022-02-08 12:37:37,2022-02-08 12:37:08,-122.404300,37.630930,POINT (-211870.039 -40160.972),21033.876226,23.0,-4499.480366,False,-195.629581
92,2022-02-08 12:41:41,2022-02-08 12:41:13,-122.390460,37.614292,POINT (-210697.329 -42040.801),28645.703043,28.0,-54.313282,False,-1.939760
151,2022-02-08 13:11:18,2022-02-08 13:11:06,-122.275590,37.520050,POINT (-200826.006 -52763.371),47534.789723,31.0,-2.183705,False,-0.070442
161,2022-02-08 13:16:19,2022-02-08 13:16:09,-122.275550,37.520050,POINT (-200822.477 -52763.456),47537.211790,29.0,-2.183705,False,-0.075300


In [111]:
distinct_positions

,header_timestamp,vehicle_timestamp,vehicle_longitude,vehicle_latitude,geometry,shape_meters,secs_from_last,meters_from_last,progressed,speed_from_last
0,2022-02-08 11:54:39,2022-02-08 11:54:06,-122.393360,37.794390,POINT (-210447.528 -22019.888),507.601271,NaN,NaN,False,NaN
1,2022-02-08 11:58:55,2022-02-08 11:58:35,-122.399890,37.788310,POINT (-211038.694 -22681.051),1482.537287,25.0,974.936016,True,38.997441
2,2022-02-08 12:02:55,2022-02-08 12:02:31,-122.406920,37.782290,POINT (-211673.745 -23334.388),2394.138605,27.0,911.601317,True,33.763012
3,2022-02-08 12:03:55,2022-02-08 12:03:40,-122.408394,37.781110,POINT (-211806.678 -23462.231),2578.537122,38.0,184.398518,True,4.852593
4,2022-02-08 12:04:55,2022-02-08 12:04:40,-122.411260,37.778854,POINT (-212065.052 -23706.535),2934.174313,31.0,355.637191,True,11.472167
...,...,...,...,...,...,...,...,...,...,...
62,2022-02-08 13:26:23,2022-02-08 13:26:02,-122.245830,37.494830,POINT (-198266.518 -55628.782),51383.120200,30.0,598.153786,True,19.938460
63,2022-02-08 13:27:39,2022-02-08 13:27:06,-122.241800,37.491642,POINT (-197919.187 -55991.496),51886.857139,33.0,503.736938,True,15.264756
64,2022-02-08 13:28:23,2022-02-08 13:28:10,-122.238620,37.489388,POINT (-197644.406 -56248.624),52264.772990,28.0,377.915852,True,13.496995
65,2022-02-08 13:30:39,2022-02-08 13:30:07,-122.233320,37.484795,POINT (-197188.589 -56770.096),52957.289869,32.0,692.516878,True,21.641152


In [118]:
distinct_positions.vehicle_longitude.isin(ex.cleaned_positions.vehicle_longitude)

0     False
1     False
2     False
3     False
4     False
      ...  
62     True
63     True
64     True
65     True
66     True
Name: vehicle_longitude, Length: 67, dtype: bool

In [115]:
ex.cleaned_positions >> anti_join(_, distinct_positions)

,header_timestamp,vehicle_timestamp,vehicle_longitude,vehicle_latitude,geometry,shape_meters,secs_from_last,meters_from_last,progressed,speed_from_last
78098,2022-02-08 12:38:37,2022-02-08 12:38:16,-122.400370,37.621113,POINT (-211551.423 -41260.713),27094.279677,124.0,1560.923086,True,12.588089
77974,2022-02-08 12:40:21,2022-02-08 12:39:43,-122.390150,37.614540,POINT (-210669.316 -42013.929),28679.490101,87.0,1585.210424,True,18.220809
78133,2022-02-08 12:42:41,2022-02-08 12:42:15,-122.387140,37.615420,POINT (-210401.606 -41922.818),29027.176258,122.0,327.159932,True,2.681639
78088,2022-02-08 12:43:41,2022-02-08 12:43:11,-122.384580,37.616960,POINT (-210171.711 -41757.355),29455.975120,56.0,428.798862,True,7.657123
78028,2022-02-08 12:44:37,2022-02-08 12:44:11,-122.387726,37.616795,POINT (-210449.396 -41768.713),29828.821036,60.0,372.845917,True,6.214099
77992,2022-02-08 12:45:37,2022-02-08 12:45:10,-122.389160,37.616108,POINT (-210577.683 -41841.876),29979.085391,59.0,150.264355,True,2.546853
77989,2022-02-08 12:46:41,2022-02-08 12:46:14,-122.391550,37.615475,POINT (-210790.065 -41906.910),30204.187579,64.0,225.102188,True,3.517222
78057,2022-02-08 12:47:37,2022-02-08 12:47:14,-122.392914,37.610554,POINT (-210924.068 -42450.749),31295.938811,60.0,1091.751232,True,18.195854
78087,2022-02-08 12:48:37,2022-02-08 12:48:14,-122.379830,37.602110,POINT (-209794.577 -43418.166),32783.095194,60.0,1487.156383,True,24.785940
78073,2022-02-08 12:50:38,2022-02-08 12:50:16,-122.354820,37.587425,POINT (-207630.812 -45105.181),35551.419623,122.0,2768.324430,True,22.691184


In [93]:
simple = distinct_positions >> select(_.geometry, _.shape_meters)

In [97]:
popup_dict = {'shape_meters':'shape_meters'}

In [101]:
shared_utils.map_utils.make_folium_choropleth_map(simple,
                                                  'shape_meters',
                                                 popup_dict,
                                                 popup_dict,
                                                 branca.colormap.step.Blues_03,
                                                 1300, 700)

In [89]:
ex.cleaned_positions.shape

(38, 10)

In [82]:
[categorize_cleaning(sam, key) for key in sam.position_interpolators.keys()]

[('11403326-137-Blocks-Weekday-14', 0.8290598290598291, 0.941747572815534),
 ('11427096-137-Blocks-Weekday-14', 0.08020050125313283, 0.8),
 ('11403562-137-Blocks-Weekday-14', 0.3333333333333333, 0.7692307692307693),
 ('11403225-137-Blocks-Weekday-14', 0.0982367758186398, 0.8297872340425532),
 ('11427080-137-Blocks-Weekday-14', 0.08024691358024691, 0.7222222222222222),
 ('11402688-137-Blocks-Weekday-14', 0.3563218390804598, 0.9207920792079208),
 ('11401868-137-Blocks-Weekday-14', 0.13402061855670103, 0.6842105263157895),
 ('11403182-137-Blocks-Weekday-14', 0.7083333333333334, 0.8947368421052632),
 ('11402784-137-Blocks-Weekday-14', 0.531578947368421, 0.8381742738589212),
 ('11402692-137-Blocks-Weekday-14', 0.4020618556701031, 0.9669421487603306),
 ('11403096-137-Blocks-Weekday-14', 0.6636363636363637, 0.9605263157894737),
 ('11402002-137-Blocks-Weekday-14', 0.4729064039408867, 0.8135593220338984),
 ('11402503-137-Blocks-Weekday-14', 0.8554216867469879, 0.9102564102564102),
 ('11402014-1

In [171]:
sam_non_progress_series = [ac.position_interpolators[key]['rt'].position_gdf >> filter(-_.progressed) >> select(_.meters_from_last) for key in ac.position_interpolators.keys()]

In [172]:
all_np = pd.Series()
for np in sam_non_progress_series:
    all_np = pd.concat((all_np, np))

/tmp/ipykernel_866/3533518072.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [173]:
all_np.meters_from_last.median()

0.0

In [174]:
all_np = all_np.apply(lambda x: abs(x)) >> filter(_.meters_from_last > 0)

In [175]:
all_np.meters_from_last.quantile(.5)

75.76255677501467

In [176]:
all_np.meters_from_last.quantile(.8)

380.14564218718806

In [177]:
all_np.meters_from_last.mean()

507.89220226628

In [178]:
all_np.meters_from_last.value_counts()

155.396068    36
20.320559     22
20.235695     20
139.294796    15
866.889192    15
              ..
116.235756     1
762.648990     1
17.318542      1
19.251011      1
42.059612      1
Name: meters_from_last, Length: 8230, dtype: int64

In [84]:
# [categorize_cleaning(ac, key) for key in ac.position_interpolators.keys()]